In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.backends.cudnn as cudnn
from torch.optim import Adam, SGD
import torchvision
import torchvision.transforms as transforms

import sys, os, math
import argparse

In [2]:
lr=0.01
data='cifar10'
root='./data/'
model='vgg'
model_out='./checkpoint/cifar10_vgg_PReLU.pth'
resume = False

In [3]:
if data == 'cifar10':
    nclass = 10
    img_width = 32
    transform_train = transforms.Compose([
#         transforms.RandomCrop(32, padding=4),
#         transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
    ])
    transform_test = transforms.Compose([
        transforms.ToTensor(),
    ])
    trainset = torchvision.datasets.CIFAR10(root=root, train=True, download=True, transform=transform_train)
    trainloader = torch.utils.data.DataLoader(trainset, batch_size=256, shuffle=True, num_workers=8)
    testset = torchvision.datasets.CIFAR10(root=root, train=False, download=True, transform=transform_test)
    testloader = torch.utils.data.DataLoader(testset, batch_size=100, shuffle=False, num_workers=8)
    
elif data == 'cifar100':
    nclass = 100
    img_width = 32
    transform_train = transforms.Compose([
#         transforms.RandomCrop(32, padding=4),
#         transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
    ])
    transform_test = transforms.Compose([
        transforms.ToTensor(),
    ])
    trainset = torchvision.datasets.CIFAR100(root=root, train=True, download=True, transform=transform_train)
    trainloader = torch.utils.data.DataLoader(trainset, batch_size=256, shuffle=True, num_workers=8)
    testset = torchvision.datasets.CIFAR100(root=root, train=False, download=True, transform=transform_test)
    testloader = torch.utils.data.DataLoader(testset, batch_size=100, shuffle=False, num_workers=8)
    
elif data == 'stl10':
    nclass = 10
    img_width = 32
    transform_train = transforms.Compose([
#         transforms.RandomCrop(32, padding=4),
#         transforms.RandomHorizontalFlip(),
        transforms.Resize((img_width,img_width)),
        transforms.ToTensor(),
    ])
    transform_test = transforms.Compose([
        transforms.Resize((img_width,img_width)),
        transforms.ToTensor(),
    ])
    trainset = torchvision.datasets.STL10(root=root, split='train', transform=transform_train, target_transform=None, download=True)
    testset = torchvision.datasets.STL10(root=root, split='test', transform=transform_test, target_transform=None, download=True)
    trainloader = torch.utils.data.DataLoader(trainset, batch_size=256, shuffle=True, num_workers=8)
    testloader = torch.utils.data.DataLoader(testset, batch_size=100, shuffle=False, num_workers=8)

Files already downloaded and verified
Files already downloaded and verified


In [4]:
if model == 'vgg':
    from models.vgg import VGG_PReLU
    net = nn.DataParallel(VGG_PReLU('VGG16', nclass, img_width=img_width).cuda())
    
net

DataParallel(
  (module): VGG_PReLU(
    (features): Sequential(
      (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): PReLU(num_parameters=1)
      (3): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (4): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (5): PReLU(num_parameters=1)
      (6): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
      (7): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (8): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (9): PReLU(num_parameters=1)
      (10): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (11): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (12): PReLU(num_parameters=1)
      (13): MaxPool2d(kernel_s

In [5]:
if resume:
    print(f'==> Resuming from {model_out}')
    net.load_state_dict(torch.load(model_out))

In [6]:
cudnn.benchmark = True

In [7]:
criterion = nn.CrossEntropyLoss()

In [8]:
def train(epoch):
    print('Epoch: %d' % epoch)
    net.train()
    train_loss = 0
    correct = 0
    total = 0
    for batch_idx, (inputs, targets) in enumerate(trainloader):
        inputs, targets = inputs.cuda(), targets.cuda()
        optimizer.zero_grad()
        outputs, _ = net(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()
        pred = torch.max(outputs, dim=1)[1]
        correct += torch.sum(pred.eq(targets)).item()
        total += targets.numel()
    print(f'[TRAIN] Acc: {100.*correct/total:.3f}')

In [9]:
def test(epoch):
    net.eval()
    test_loss = 0
    correct = 0
    total = 0
    with torch.no_grad():
        for batch_idx, (inputs, targets) in enumerate(testloader):
            inputs, targets = inputs.cuda(), targets.cuda()
            outputs, _ = net(inputs)
            loss = criterion(outputs, targets)
            test_loss += loss.item()
            _, predicted = outputs.max(1)
            total += targets.size(0)
            correct += predicted.eq(targets).sum().item()
        print(f'[TEST] Acc: {100.*correct/total:.3f}')

    # Save checkpoint after each epoch
    torch.save(net.state_dict(), model_out)

In [10]:
if data == 'cifar10':
    epochs = [1]
elif data == 'cifar100':
    epochs = [50, 50, 50, 50]
elif data == 'stl10':
    epochs = [50, 50, 50, 50]

In [11]:
count = 0

In [13]:
for epoch in epochs:
    optimizer = Adam(net.parameters(), lr=lr)
    for _ in range(epoch):
        %time train(count)
        %time test(count)
        count += 1
    lr /= 10

Epoch: 1
[TRAIN] Acc: 42.098
CPU times: user 10.1 s, sys: 3.65 s, total: 13.8 s
Wall time: 13.9 s
[TEST] Acc: 46.730
CPU times: user 727 ms, sys: 490 ms, total: 1.22 s
Wall time: 1.33 s


In [2]:
lr=0.01
data='stl10'
root='./data/'
model='vgg'
model_out='./checkpoint/stl10_vgg_PReLU.pth'
resume = False

In [12]:
for epoch in epochs:
    optimizer = Adam(net.parameters(), lr=lr)
    for _ in range(epoch):
        train(count)
        test(count)
        count += 1
    lr /= 10

Epoch: 0
[TRAIN] Acc: 11.380
[TEST] Acc: 13.988
Epoch: 1
[TRAIN] Acc: 13.000
[TEST] Acc: 16.913
Epoch: 2
[TRAIN] Acc: 15.300
[TEST] Acc: 16.788
Epoch: 3
[TRAIN] Acc: 17.840
[TEST] Acc: 11.162
Epoch: 4
[TRAIN] Acc: 20.760
[TEST] Acc: 19.288
Epoch: 5
[TRAIN] Acc: 24.000
[TEST] Acc: 18.587
Epoch: 6
[TRAIN] Acc: 27.640
[TEST] Acc: 24.637
Epoch: 7
[TRAIN] Acc: 30.240
[TEST] Acc: 32.062
Epoch: 8
[TRAIN] Acc: 34.420
[TEST] Acc: 32.138
Epoch: 9
[TRAIN] Acc: 34.820
[TEST] Acc: 28.400
Epoch: 10
[TRAIN] Acc: 36.380
[TEST] Acc: 34.675
Epoch: 11
[TRAIN] Acc: 38.660
[TEST] Acc: 33.425
Epoch: 12
[TRAIN] Acc: 40.900
[TEST] Acc: 40.638
Epoch: 13
[TRAIN] Acc: 43.620
[TEST] Acc: 34.862
Epoch: 14
[TRAIN] Acc: 47.300
[TEST] Acc: 35.312
Epoch: 15
[TRAIN] Acc: 48.200
[TEST] Acc: 30.962
Epoch: 16
[TRAIN] Acc: 51.420
[TEST] Acc: 33.225
Epoch: 17
[TRAIN] Acc: 54.340
[TEST] Acc: 31.212
Epoch: 18
[TRAIN] Acc: 57.280
[TEST] Acc: 44.125
Epoch: 19
[TRAIN] Acc: 61.300
[TEST] Acc: 45.462
Epoch: 20
[TRAIN] Acc: 64.320


[TEST] Acc: 59.837
Epoch: 165
[TRAIN] Acc: 100.000
[TEST] Acc: 59.913
Epoch: 166
[TRAIN] Acc: 100.000
[TEST] Acc: 59.975
Epoch: 167
[TRAIN] Acc: 100.000
[TEST] Acc: 60.050
Epoch: 168
[TRAIN] Acc: 100.000
[TEST] Acc: 59.900
Epoch: 169
[TRAIN] Acc: 100.000
[TEST] Acc: 59.875
Epoch: 170
[TRAIN] Acc: 100.000
[TEST] Acc: 59.712
Epoch: 171
[TRAIN] Acc: 100.000
[TEST] Acc: 59.763
Epoch: 172
[TRAIN] Acc: 100.000
[TEST] Acc: 59.775
Epoch: 173
[TRAIN] Acc: 100.000
[TEST] Acc: 59.825
Epoch: 174
[TRAIN] Acc: 100.000
[TEST] Acc: 59.888
Epoch: 175
[TRAIN] Acc: 100.000
[TEST] Acc: 59.888
Epoch: 176
[TRAIN] Acc: 100.000
[TEST] Acc: 59.862
Epoch: 177
[TRAIN] Acc: 100.000
[TEST] Acc: 60.000
Epoch: 178
[TRAIN] Acc: 100.000
[TEST] Acc: 59.888
Epoch: 179
[TRAIN] Acc: 100.000
[TEST] Acc: 59.788
Epoch: 180
[TRAIN] Acc: 100.000
[TEST] Acc: 59.750
Epoch: 181
[TRAIN] Acc: 100.000
[TEST] Acc: 59.825
Epoch: 182
[TRAIN] Acc: 100.000
[TEST] Acc: 59.950
Epoch: 183
[TRAIN] Acc: 100.000
[TEST] Acc: 59.975
Epoch: 184
[

In [2]:
lr=0.01
data='cifar100'
root='./data/'
model='vgg'
model_out='./checkpoint/cifar100_vgg_PReLU.pth'
resume = False

In [13]:
for epoch in epochs:
    optimizer = Adam(net.parameters(), lr=lr)
    for _ in range(epoch):
        train(count)
        test(count)
        count += 1
    lr /= 10

Epoch: 0
[TRAIN] Acc: 3.426
[TEST] Acc: 3.960
Epoch: 1
[TRAIN] Acc: 5.782
[TEST] Acc: 6.310
Epoch: 2
[TRAIN] Acc: 9.578
[TEST] Acc: 6.440
Epoch: 3
[TRAIN] Acc: 13.704
[TEST] Acc: 12.660
Epoch: 4
[TRAIN] Acc: 18.472
[TEST] Acc: 15.680
Epoch: 5
[TRAIN] Acc: 22.984
[TEST] Acc: 21.550
Epoch: 6
[TRAIN] Acc: 27.154
[TEST] Acc: 20.270
Epoch: 7
[TRAIN] Acc: 30.962
[TEST] Acc: 23.910
Epoch: 8
[TRAIN] Acc: 35.440
[TEST] Acc: 18.690
Epoch: 9
[TRAIN] Acc: 39.474
[TEST] Acc: 30.100
Epoch: 10
[TRAIN] Acc: 43.768
[TEST] Acc: 27.830
Epoch: 11
[TRAIN] Acc: 47.416
[TEST] Acc: 29.520
Epoch: 12
[TRAIN] Acc: 51.422
[TEST] Acc: 32.580
Epoch: 13
[TRAIN] Acc: 55.230
[TEST] Acc: 32.330
Epoch: 14
[TRAIN] Acc: 59.860
[TEST] Acc: 32.610
Epoch: 15
[TRAIN] Acc: 62.998
[TEST] Acc: 34.090
Epoch: 16
[TRAIN] Acc: 67.052
[TEST] Acc: 35.710
Epoch: 17
[TRAIN] Acc: 70.554
[TEST] Acc: 30.890
Epoch: 18
[TRAIN] Acc: 73.552
[TEST] Acc: 33.040
Epoch: 19
[TRAIN] Acc: 76.548
[TEST] Acc: 34.830
Epoch: 20
[TRAIN] Acc: 79.182
[TEST]

[TRAIN] Acc: 99.978
[TEST] Acc: 43.320
Epoch: 167
[TRAIN] Acc: 99.978
[TEST] Acc: 43.220
Epoch: 168
[TRAIN] Acc: 99.986
[TEST] Acc: 43.010
Epoch: 169
[TRAIN] Acc: 99.984
[TEST] Acc: 43.100
Epoch: 170
[TRAIN] Acc: 99.982
[TEST] Acc: 43.190
Epoch: 171
[TRAIN] Acc: 99.976
[TEST] Acc: 43.040
Epoch: 172
[TRAIN] Acc: 99.972
[TEST] Acc: 43.310
Epoch: 173
[TRAIN] Acc: 99.994
[TEST] Acc: 43.260
Epoch: 174
[TRAIN] Acc: 99.976
[TEST] Acc: 43.130
Epoch: 175
[TRAIN] Acc: 99.978
[TEST] Acc: 43.100
Epoch: 176
[TRAIN] Acc: 99.972
[TEST] Acc: 43.130
Epoch: 177
[TRAIN] Acc: 99.978
[TEST] Acc: 43.180
Epoch: 178
[TRAIN] Acc: 99.986
[TEST] Acc: 42.980
Epoch: 179
[TRAIN] Acc: 99.978
[TEST] Acc: 43.300
Epoch: 180
[TRAIN] Acc: 99.982
[TEST] Acc: 43.100
Epoch: 181
[TRAIN] Acc: 99.976
[TEST] Acc: 43.200
Epoch: 182
[TRAIN] Acc: 99.984
[TEST] Acc: 43.070
Epoch: 183
[TRAIN] Acc: 99.986
[TEST] Acc: 43.140
Epoch: 184
[TRAIN] Acc: 99.982
[TEST] Acc: 43.040
Epoch: 185
[TRAIN] Acc: 99.986
[TEST] Acc: 43.120
Epoch: 186
